# 4.6 Revisión de Código con IA (Pair Programming)

- Objetivo: usar IA como revisor de código para detectar errores, sugerir mejoras de estilo y verificar buenas prácticas.
- Prerrequisitos: código SQL funcional que necesita revisión.
- Método: proporcionar query completa + checklist de revisión → solicitar análisis punto por punto.
- Aplicaciones: code review automatizado, onboarding de juniors, standarización de equipo.

## Ejemplo Práctico

**Prompt sugerido:**
```
Revisa esta consulta según checklist de calidad:
1. Legibilidad (nombres, indentación)
2. Performance (índices, JOINs eficientes)
3. Manejo de NULLs
4. Seguridad (SQL injection si es dinámico)
5. Documentación

Consulta:
SELECT c.nombre,sum(v.cantidad*p.precio_unitario) AS total FROM fact_ventas v
JOIN dim_clientes c ON v.cliente_id=c.cliente_id JOIN dim_productos p ON v.producto_id=p.producto_id
WHERE fecha>='2024-01-01' GROUP BY c.nombre;
```

**Respuesta esperada de la IA:**
"**Revisión de Código:**
1. ✅ **Legibilidad**: usar saltos de línea, alias explícitos (`v.cantidad` está bien pero falta `dbo.`), indentar JOINs
2. ⚠️ **Performance**: considerar índice en `fact_ventas(fecha, cliente_id, producto_id)`, verificar si `GROUP BY c.cliente_id` es más eficiente que por nombre
3. ⚠️ **NULLs**: si `descuento_pct` puede ser NULL, agregar `ISNULL(v.descuento_pct, 0)` en cálculo
4. ✅ **Seguridad**: consulta estática, no hay riesgo de injection
5. ❌ **Documentación**: agregar comentario explicando objetivo (total ventas por cliente en 2024)

**Código mejorado:**
```sql
-- Total de ingresos por cliente en 2024
SELECT c.cliente_id, c.nombre,
       SUM(v.cantidad * p.precio_unitario * (1 - ISNULL(v.descuento_pct,0)/100.0)) AS ingresos_netos
FROM dbo.fact_ventas v
INNER JOIN dbo.dim_clientes c ON v.cliente_id = c.cliente_id
INNER JOIN dbo.dim_productos p ON v.producto_id = p.producto_id
WHERE v.fecha >= '2024-01-01' AND v.fecha < '2025-01-01'
GROUP BY c.cliente_id, c.nombre
ORDER BY ingresos_netos DESC;
-- Índice sugerido: CREATE INDEX idx_ventas_fecha ON fact_ventas(fecha) INCLUDE (cliente_id, producto_id, cantidad, descuento_pct);
```"

---

## 🟢 Ejercicio Básico
Solicita revisión de legibilidad de una consulta sin formato (todo en una línea).

## 🟠 Ejercicio Intermedio
Pide análisis de performance de consulta con 3 subconsultas correlacionadas.

## 🔴 Ejercicio Avanzado
Solicita revisión de SP dinámico con EXEC(@sql) verificando vulnerabilidades de SQL injection.

---

## Errores Comunes
- No especificar checklist: IA puede enfocarse solo en sintaxis y omitir performance
- Aceptar todas las sugerencias: evaluar impacto real (ej. índice sugerido puede ser redundante)
- No iterar: hacer revisión en múltiples pasadas (primero legibilidad, luego performance)

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[⬅️ Anterior](05_generacion_datos_sinteticos.ipynb) | [Siguiente ➡️](07_etica_y_gobernanza_ia.ipynb)
---
